Mục tiêu: Xây dựng một lớp Python để thực hiện mã hóa và giải mã cho ba loại mật mã: Caesar cipher, Affine cipher, và Exponentiation cipher.

Yêu cầu:

1.	Thuộc tính: Không cần.

2.	Phương thức mã hóa và giải mã:

    a.	Caesar Cipher:

    -	caesar_encrypt(plaintext, shift): xem section 8.1

    -	caesar_decrypt(ciphertext, shift): xem section 8.1

    b.	Affine Cipher:

    -	affine_encrypt(plaintext, a, b): xem section 8.1

    -	affine_decrypt(ciphertext, a, b): xem section 8.1

    c.	Exponentiation Cipher:

    -	exponentiation_encrypt(plaintext, e, n): xem section 8.3

    -	exponentiation_decrypt(ciphertext, d, n): xem section 8.3

3.	Chạy test cases:

-	Case 1: Sử dụng đoạn văn bản plaintext = " LIFE IS A DREAM" với shift = 3 cho Caesar cipher.

-	Case 2: Sử dụng đoạn văn bản plaintext = " LIFE IS A DREAM" với a = 5 và b = 8 cho Affine cipher.

-	Case 3: Sử dụng đoạn văn bản plaintext = " LIFE IS A DREAM" với e = 43, d=viết hàm tính, và n = 2633 cho Exponentiation cipher.

In [179]:
import math
from itertools import cycle, pairwise
from collections import Counter # Use for counting frequency

alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
tabula_recta = {char: alphabet[shift:] + alphabet[:shift] 
                for shift, char in enumerate(alphabet)}
tabula_recta

{'A': 'ABCDEFGHIJKLMNOPQRSTUVWXYZ',
 'B': 'BCDEFGHIJKLMNOPQRSTUVWXYZA',
 'C': 'CDEFGHIJKLMNOPQRSTUVWXYZAB',
 'D': 'DEFGHIJKLMNOPQRSTUVWXYZABC',
 'E': 'EFGHIJKLMNOPQRSTUVWXYZABCD',
 'F': 'FGHIJKLMNOPQRSTUVWXYZABCDE',
 'G': 'GHIJKLMNOPQRSTUVWXYZABCDEF',
 'H': 'HIJKLMNOPQRSTUVWXYZABCDEFG',
 'I': 'IJKLMNOPQRSTUVWXYZABCDEFGH',
 'J': 'JKLMNOPQRSTUVWXYZABCDEFGHI',
 'K': 'KLMNOPQRSTUVWXYZABCDEFGHIJ',
 'L': 'LMNOPQRSTUVWXYZABCDEFGHIJK',
 'M': 'MNOPQRSTUVWXYZABCDEFGHIJKL',
 'N': 'NOPQRSTUVWXYZABCDEFGHIJKLM',
 'O': 'OPQRSTUVWXYZABCDEFGHIJKLMN',
 'P': 'PQRSTUVWXYZABCDEFGHIJKLMNO',
 'Q': 'QRSTUVWXYZABCDEFGHIJKLMNOP',
 'R': 'RSTUVWXYZABCDEFGHIJKLMNOPQ',
 'S': 'STUVWXYZABCDEFGHIJKLMNOPQR',
 'T': 'TUVWXYZABCDEFGHIJKLMNOPQRS',
 'U': 'UVWXYZABCDEFGHIJKLMNOPQRST',
 'V': 'VWXYZABCDEFGHIJKLMNOPQRSTU',
 'W': 'WXYZABCDEFGHIJKLMNOPQRSTUV',
 'X': 'XYZABCDEFGHIJKLMNOPQRSTUVW',
 'Y': 'YZABCDEFGHIJKLMNOPQRSTUVWX',
 'Z': 'ZABCDEFGHIJKLMNOPQRSTUVWXY'}

In [180]:
# Reimplement the batched() function from itertools in Python
from itertools import islice
def batched(iterable, n):
    """Yield successive n-sized chunks from the iterable."""
    it = iter(iterable)
    while True:
        batch = list(islice(it, n))
        if not batch:
            break
        yield batch

In [181]:
class Cryptography:
    # Caesar Cipher
    @staticmethod
    def create_caesar_dict(shift):
        shift = shift % 26
        shifted_alphabet = alphabet[shift:] + alphabet[:shift] # right rotate
        # shifted_alphabet = alphabet[-shift:] + alphabet[:-shift] # left rotate
        return {char: shifted 
                for char, shifted in zip(alphabet, shifted_alphabet)}

    @staticmethod
    def caesar_encrypt(plaintext, shift):
        cipher_dict = Cryptography.create_caesar_dict(shift)
        encrypted = ''.join(cipher_dict.get(char, char) for char in plaintext)
        return encrypted
    
    @staticmethod
    def caesar_decrypt(ciphertext, shift):
        decipher_dict = Cryptography.create_caesar_dict(-shift)
        decrypted = ''.join(decipher_dict.get(char, char) for char in ciphertext)
        return decrypted
    
    @staticmethod
    def caesar_breaking_brute_force(ciphertext):
        return {shift: Cryptography.caesar_decrypt(ciphertext, shift) 
                for shift in range(1, 27)}
    
    @staticmethod
    def caesar_breaking_frequency_analysis(ciphertext):
        # Common English letter frequency
        most_frequency = 'ETAOINSHRDLCUMWFGYPBVKJXQZ'[:8]
        # most_frequency = 'ETAOIN'
        chars_frequency = Counter(ciphertext.replace(' ', ''))
        most_common_char = chars_frequency.most_common(1)[0][0]
        print(f'Most common charater in ciphertext: {most_common_char}')
        result = {}
        for char in most_frequency:
            shift = alphabet.index(most_common_char) - alphabet.index(char)
            result[shift % 26] = Cryptography.caesar_decrypt(ciphertext, shift)
        return result

    # Affine Cipher
    @staticmethod
    def affine_encrypt(plaintext, a, b):
        transformed_index = [(a*c + b) % 26 for c in list(range(26))]
        transformed_alphabet = [alphabet[index] for index in transformed_index]
        cipher_dict = {char: transformed 
                       for char, transformed in zip(alphabet, transformed_alphabet)}
        encrypted = ''.join(cipher_dict.get(char, char) for char in plaintext)
        return encrypted

    @staticmethod
    def affine_decrypt(ciphertext, a, b):
        a_inv = pow(a, -1, 26)  # Modular multiplicative inverse of a mod 26
        transformed_index = [a_inv * (c-b) % 26 for c in list(range(26))]
        transformed_alphabet = [alphabet[i] for i in transformed_index]
        cipher_dict = {char: transformed 
                       for char, transformed in zip(alphabet, transformed_alphabet)}
        decrypted = ''.join(cipher_dict.get(char, char) for char in ciphertext)
        return decrypted

    @staticmethod
    def affine_breaking_brute_force(ciphertext):
        possible_plaintexts = {}
        # Relative primes of 26
        relative_primes = [a for a in range(26) if math.gcd(a, 26) == 1]
        # 312 possible keys
        for a in relative_primes:
            for b in range(26):
                decrypted = Cryptography.affine_decrypt(ciphertext, a, b)
                possible_plaintexts[f'{a = }, {b = }'] = decrypted
        return possible_plaintexts        

    @staticmethod
    def affine_breaking_frequency_analysis(ciphertext):
        result = {}
        most_frequency = 'ETAOINSHRDLCUMWFGYPBVKJXQZ'[:8]
        # most_frequency = 'ETAOIN'
        chars_frequency = Counter(ciphertext.replace(' ', ''))
        most_common_char = [char for char, _ in chars_frequency.most_common()]
        for pair in pairwise(most_common_char):
            b1, b2 = [alphabet.index(char) for char in pair]
            for pair_frequency in pairwise(most_frequency):
                a1, a2 = [alphabet.index(char) for char in pair_frequency]
                try:
                    a = (a2 - a1) % 26
                    b = (b2 - b1) % 26
                    a_inv = pow(a, -1, 26)
                    a = pow(b * a_inv, -1, 26)
                    b = (b1 - a1*a) % 26
                    result[f'{a = }, {b = }'] = Cryptography.affine_decrypt(ciphertext, a, b)
                except:
                    pass

        return result

    # Exponentiation Cipher
    @staticmethod
    def exponentiation_encrypt(plaintext, e, p):
        m = 1
        while not (int(str(25)*m) < p and p < int(str(25)*(m+1))):
            m += 1 

        encrypted = []
        plaintext = plaintext.replace(' ', '')

        for batch in batched(plaintext, m):
            P = ''.join(f'{alphabet.index(char):02}' for char in batch)

            while len(P) < 2 * m:
                P += '23'

            c = pow(int(P), e, p)
            encrypted.append(str(c).rjust(2*m, '0'))

        return encrypted

    @staticmethod
    def exponentiation_decrypt(ciphertext, d, p):
        decrypted = ''
        for char in ciphertext:
            P = str(pow(int(char), d, p)).rjust(len(char), '0')
            while len(P) > 1:
                decrypted += alphabet[int(P[:2])]
                P = P[2:]

        return decrypted
    
    # Vigenere Cipher
    @staticmethod
    def vigenere_encrypt(plaintext, key):
        keys = cycle(key)
        encrypted = ''
        for char in plaintext:
            current_key = next(keys)
            if char not in alphabet:
                encrypted += char
                continue
            encrypted += tabula_recta[current_key][alphabet.index(char)]
        return encrypted
    
    @staticmethod
    def vigenere_decrypt(ciphertext, key):
        keys = cycle(key)
        decrypted = ''
        for char in ciphertext:
            current_key = next(keys)
            if char not in alphabet:
                encrypted += char
                continue
            decrypted += alphabet[tabula_recta[current_key].index(char)]
        return decrypted

    # Autokey cipher
    @staticmethod
    def autokey_encrypt(plaintext, seed):
        keystream = [alphabet.index(seed)]
        keystream.extend([alphabet.index(p) for p in plaintext])
        encrypted = ''
        for k, p in zip(keystream, plaintext):
            encrypted += alphabet[(alphabet.index(p) + k) % 26]
        return encrypted
    
    @staticmethod    
    def autokey_decrypt(ciphertext, seed):
        s = alphabet.index(seed)
        decrypted = ''
        for c in ciphertext:
            s = (alphabet.index(c) - s) % 26
            decrypted += alphabet[s]

        return decrypted

## Case 1

In [182]:
# plaintext = 'this message is top secret'
plaintext = 'LIFE IS A DREAM'
shift = 3
print(f'{plaintext = }')
encrypted = Cryptography.caesar_encrypt(plaintext, shift)
print(f'{encrypted = }')
decrypted = Cryptography.caesar_decrypt(encrypted, shift)
print(f'{decrypted = }')

plaintext = 'LIFE IS A DREAM'
encrypted = 'OLIH LV D GUHDP'
decrypted = 'LIFE IS A DREAM'


### Brute force

In [183]:
Cryptography.caesar_breaking_brute_force(encrypted)

{1: 'NKHG KU C FTGCO',
 2: 'MJGF JT B ESFBN',
 3: 'LIFE IS A DREAM',
 4: 'KHED HR Z CQDZL',
 5: 'JGDC GQ Y BPCYK',
 6: 'IFCB FP X AOBXJ',
 7: 'HEBA EO W ZNAWI',
 8: 'GDAZ DN V YMZVH',
 9: 'FCZY CM U XLYUG',
 10: 'EBYX BL T WKXTF',
 11: 'DAXW AK S VJWSE',
 12: 'CZWV ZJ R UIVRD',
 13: 'BYVU YI Q THUQC',
 14: 'AXUT XH P SGTPB',
 15: 'ZWTS WG O RFSOA',
 16: 'YVSR VF N QERNZ',
 17: 'XURQ UE M PDQMY',
 18: 'WTQP TD L OCPLX',
 19: 'VSPO SC K NBOKW',
 20: 'URON RB J MANJV',
 21: 'TQNM QA I LZMIU',
 22: 'SPML PZ H KYLHT',
 23: 'ROLK OY G JXKGS',
 24: 'QNKJ NX F IWJFR',
 25: 'PMJI MW E HVIEQ',
 26: 'OLIH LV D GUHDP'}

### Frequency analysis

In [184]:
Cryptography.caesar_breaking_frequency_analysis(encrypted)

Most common charater in ciphertext: L


{7: 'HEBA EO W ZNAWI',
 18: 'WTQP TD L OCPLX',
 11: 'DAXW AK S VJWSE',
 23: 'ROLK OY G JXKGS',
 3: 'LIFE IS A DREAM',
 24: 'QNKJ NX F IWJFR',
 19: 'VSPO SC K NBOKW',
 4: 'KHED HR Z CQDZL'}

## Case 2

In [185]:
# plaintext = 'PLEASE SEND MONEY'
# a, b = 7, 10
plaintext = 'LIFE IS A DREAM'
a, b = 5, 8
print(f'{plaintext = }')
encrypted = Cryptography.affine_encrypt(plaintext, a, b)
print(f'{encrypted = }')
decrypted = Cryptography.affine_decrypt(encrypted, a, b)
print(f'{decrypted = }')

plaintext = 'LIFE IS A DREAM'
encrypted = 'LWHC WU I XPCIQ'
decrypted = 'LIFE IS A DREAM'


### Brute force

In [186]:
# Cryptography.affine_breaking_brute_force(encrypted)

### Frequency analysis

In [187]:
Cryptography.affine_breaking_frequency_analysis(encrypted)

{'a = 5, b = 14': 'PMJI MW E HVIEQ',
 'a = 11, b = 7': 'YZAJ ZN T SWJTP',
 'a = 19, b = 12': 'PGXU GK I RHUIS',
 'a = 19, b = 21': 'ULCZ LP N WMZNX',
 'a = 5, b = 22': 'DAXW AK S VJWSE',
 'a = 5, b = 0': 'XURQ UE M PDQMY',
 'a = 11, b = 19': 'EFGP FT Z YCPZV',
 'a = 19, b = 24': 'NEVS EI G PFSGQ',
 'a = 19, b = 7': 'SJAX JN L UKXLV',
 'a = 5, b = 8': 'LIFE IS A DREAM',
 'a = 15, b = 7': 'CBAR BN H IERHL',
 'a = 7, b = 12': 'LUDG UQ S JTGSI',
 'a = 5, b = 1': 'CZWV ZJ R UIVRD',
 'a = 5, b = 2': 'HEBA EO W ZNAWI',
 'a = 15, b = 5': 'QPOF PB V WSFVZ'}

## Case 3

In [188]:
plaintext = 'LIFE IS A DREAM'
e = 43
p = 2633
print(f'{plaintext = }')
encrypted = Cryptography.exponentiation_encrypt(plaintext, e, p)
print(f'{encrypted = }')

d = pow(e, -1, p-1)
print(f'{d = }')
decrypted = Cryptography.exponentiation_decrypt(encrypted, d, p)
print(f'{decrypted = }')

plaintext = 'LIFE IS A DREAM'
encrypted = ['0894', '2373', '0953', '1022', '2559', '0798']
d = 1163
decrypted = 'LIFEISADREAM'


In [189]:
# This example was taken from the textbook
plaintext = 'THIS IS AN EXAMPLE OF AN EXPONENTIATION CIPHER'
p = 2633
e = 29

print(f'{plaintext = }')
encrypted = Cryptography.exponentiation_encrypt(plaintext, e, p)
print(f'{encrypted = }')

d = pow(e, -1, p-1)
print(f'{d = }')
decrypted = Cryptography.exponentiation_decrypt(encrypted, d, p)
print(f'{decrypted = }')

plaintext = 'THIS IS AN EXAMPLE OF AN EXPONENTIATION CIPHER'
encrypted = ['2199', '1745', '1745', '1206', '2437', '2425', '1729', '1619', '0935', '0960', '1072', '1541', '1701', '1553', '0735', '2064', '1351', '1704', '1841', '1459']
d = 2269
decrypted = 'THISISANEXAMPLEOFANEXPONENTIATIONCIPHERX'


## Vigenère Cipher

In [190]:
# Exampes were taken from the textbook
plaintext = 'MILLENNIUM'
key = 'YTWOK'
print(f'{plaintext = }')
encrypted = Cryptography.vigenere_encrypt(plaintext, key)
print(f'{encrypted = }')
decrypted = Cryptography.vigenere_decrypt(encrypted, key)
print(f'{decrypted = }')

plaintext = 'MILLENNIUM'
encrypted = 'KBHZOLGEIW'
decrypted = 'MILLENNIUM'


In [191]:
Cryptography.vigenere_decrypt('FFFLBCVFX', 'ZORRO')

'GROUNDHOG'

## Autokey Cipher

In [192]:
# Exampes were taken from the textbook
plaintext = 'HERMIT'
seed = 'X'
Cryptography.autokey_encrypt(plaintext, seed)

'ELVDUB'

In [193]:
# Exampes were taken from the textbook
ciphertext = 'RMNTU'
seed = 'F'
Cryptography.autokey_decrypt(ciphertext, seed)

'MANGO'